In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Define the ResNet architecture
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(residual)
        out = F.relu(out)
        return out

In [3]:
class ResNet(nn.Module):
    def __init__(self, num_classes=14):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self.make_layer(ResidualBlock, 16, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 32, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 64, 2, stride=2)
        self.dropout = nn.Dropout(0.5)  # Dropout added for regularization
        self.fc = None # Initialize fc layer as None

    def make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        if self.fc is None:
            self.fc = nn.Linear(out.size(1), 14)  # Initialize fc layer with correct input features
        out =self.fc(out)
        return out

In [4]:
# Define the GCN part with additional normalization (unchanged)
class GCN(nn.Module):
    def __init__(self, in_features: int, out_features: int):
        super(GCN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input: torch.Tensor, adj: torch.Tensor, deg: torch.Tensor):
        epsilon = 1e-9
        input = input.matmul(self.weight)
        deg = torch.diag(deg.sum(dim=1)) + epsilon
        deg_inv_sqrt = torch.pow(deg, 0.5)
        deg_inv_sqrt[torch.isinf(deg_inv_sqrt)] = 0
        output = deg_inv_sqrt.matmul(adj).matmul(deg_inv_sqrt).matmul(input.transpose(0, 1))
        output = output.transpose(0, 1)
        return output


In [5]:
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adj):
        x = self.linear(x)
        x = torch.matmul(x, adj)  # Graph convolution operation
#        x = F.relu(x)  # Activation function
        return x
    
class GCN2(nn.Module):
    def __init__(self, in_features, hidden_dim, out_features):
        super(GCN2, self).__init__()
        self.gcn1 = GCNLayer(in_features, hidden_dim)
        self.gcn2 = GCNLayer(hidden_dim, out_features)

    def forward(self, x, adj):
        x = self.gcn1(x, adj)
        x = self.gcn2(x, adj)
        return x

In [6]:
class FedGNN(nn.Module):
    def __init__(self, resnet: ResNet, gcn: GCN, gcn2:GCN2):
        super(FedGNN, self).__init__()
        self.resnet = resnet
        self.gcn = gcn
        self.gcn2 = gcn2

    def forward(self, x: torch.Tensor, adj: torch.Tensor, deg: torch.Tensor):
        x = self.resnet(x)
        x = self.gcn(x, adj, deg)
        x = self.gcn2(x,adj)
        return x

In [7]:
# Rest of the code remains the same
def load_data_and_split(data_dir: str, transform: transforms.Compose, train_split=0.6, val_split=0.2):
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    total_size = len(dataset)
    train_size = int(total_size * train_split)
    val_size = int(total_size * val_split)
    test_size = total_size - train_size - val_size
    print(total_size,train_size,val_size,test_size)
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    return train_dataset, val_dataset, test_dataset

# Transformations
transformations = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and split the data
train_dataset, val_dataset, test_dataset = load_data_and_split('/kaggle/input/plant-dis/PLANT DISEASES/PD', transformations)

# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Model, criterion, optimizer, and scheduler setup
# Assume the ResNet's output shape is 14 (adjust this value if it's different)
resnet = ResNet()
gcn = GCN(in_features=14, out_features=14)
gcn2 = GCN2(in_features=14, hidden_dim = 14,out_features=14)
model = FedGNN(resnet,gcn, gcn2)
#model = FedGNN(ResNet(), GCN(in_features=64, out_features=14))



criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.0001)  # Learning rate decay
adj_matrix = torch.ones(14) - torch.eye(14) 
row_sums = adj_matrix.sum(dim=1)
adj_matrix_normalized = adj_matrix / row_sums.unsqueeze(1) # Simplified adjacency matrix, replace with your actual adjacency matrix
degree_matrix = torch.diag(adj_matrix.sum(1))

2735 1641 547 547


In [8]:
print(len(train_loader))
print(len(val_loader))
print(len(test_loader))

52
18
18


In [9]:
print(model)

FedGNN(
  (resnet): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): ResidualBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       

In [10]:
# Training, validation, and testing functions
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images,adj_matrix, degree_matrix)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        scheduler.step()  # Step the scheduler at each epoch
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

def validate_model(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images,adj_matrix, degree_matrix)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    average_loss = total_loss / len(val_loader)
    print(f'Validation Loss: {average_loss}')

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images,adj_matrix, degree_matrix)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test images: {100 * correct / total}%')


In [11]:
# Execute the training, validation, and testing
train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=10)
validate_model(model, val_loader, criterion)
test_model(model, test_loader)


Epoch 1, Loss: 6.028417587280273
Epoch 2, Loss: 3.9241180419921875
Epoch 3, Loss: 2.203598737716675
Epoch 4, Loss: 3.6041746139526367
Epoch 5, Loss: 3.8227932453155518
Epoch 6, Loss: 4.061814308166504
Epoch 7, Loss: 8.737133026123047
Epoch 8, Loss: 2.2948732376098633
Epoch 9, Loss: 3.559375286102295
Epoch 10, Loss: 6.660406112670898
Validation Loss: 2.186882926358117
Accuracy of the model on the test images: 36.01462522851919%


In [12]:
test_model(model, test_loader)

Accuracy of the model on the test images: 36.01462522851919%
